In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

In [5]:
!wget -x "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8317/109592/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1602773811&Signature=dRRvrDO%2BI2DUkR3H5xqRMy24bSoTuoYuFZgw1CcCLkHL%2BZV3AWT5rwolrlUzSmwaEi2mgDbvdBt9paXat6uk8ZHVuXmUamyhfT9H7iO5zSa5KLvivFmt00t2xTeFeR9Q1snKOi04qHWjhnWo80qP6cyW25Jpglvv46r9V4B5QPikVjkIVtlczHpsP9qDgHRut%2FER4V2MzeEyyfGX%2BYec75x03aFFJEZ1IMWF2usd7BRYm0Irs6LZuWkJ9krI2%2BkH44lf%2B7HNmj%2BMe9DN%2F8ZQKpoQlELa%2FiI01ebr5b349FhKbF53S2NFSv4Ihiw44QyPjow%2BTPWXx%2FrvBXcWez8TrA%3D%3D&response-content-disposition=attachment%3B+filename%3Dfake-news.zip" `-O data.zip`
!unzip `data.zip`

/bin/bash: -O: command not found
The name is too long, 596 chars total.
Trying to shorten...
New name is storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8317/109592/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1602773811&Signature=dRRvrDO+I2DUkR3H5xqRMy24bSoTuoYuFZgw1CcCLkHL+ZV3AWT5rwol.
--2020-10-13 11:41:11--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/8317/109592/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1602773811&Signature=dRRvrDO%2BI2DUkR3H5xqRMy24bSoTuoYuFZgw1CcCLkHL%2BZV3AWT5rwolrlUzSmwaEi2mgDbvdBt9paXat6uk8ZHVuXmUamyhfT9H7iO5zSa5KLvivFmt00t2xTeFeR9Q1snKOi04qHWjhnWo80qP6cyW25Jpglvv46r9V4B5QPikVjkIVtlczHpsP9qDgHRut%2FER4V2MzeEyyfGX%2BYec75x03aFFJEZ1IMWF2usd7BRYm0Irs6LZuWkJ9krI2%2BkH44lf%2B7HNmj%2BMe9DN%2F8ZQKpoQlELa%2FiI01ebr5b349FhKbF53S2NFSv4Ihiw44QyPjow%2BTPWXx%2FrvBXcWez8TrA%3D%3D&response-content-disposition=attachment%3B+filename%3Dfake-news.zip
Resol

In [3]:
df=pd.read_csv('train.csv')
conversion_dict = {0: 'Real', 1: 'Fake'}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()

FileNotFoundError: ignored

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], df['label'], test_size=0.25, random_state=7, shuffle=True)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75)

In [ ]:
vec_train=tfidf_vectorizer.fit_transform(x_train.values.astype('U')) 
vec_test=tfidf_vectorizer.transform(x_test.values.astype('U'))

In [ ]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(vec_train,y_train)

In [ ]:
y_pred=pac.predict(vec_test)
score=accuracy_score(y_test,y_pred)
print(f'PAC Accuracy: {round(score*100,2)}%')

In [ ]:
confusion_matrix(y_test,y_pred, labels=['Real','Fake'])

In [ ]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))

In [ ]:
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'K Fold Accuracy: {round(scores.mean()*100,2)}%')

In [ ]:
df_true=pd.read_csv('True.csv')
df_true['label']='Real'
df_true_rep=[df_true['text'][i].replace('WASHINGTON (Reuters) - ','').replace('LONDON (Reuters) - ','').replace('(Reuters) - ','') for i in range(len(df_true['text']))]
df_true['text']=df_true_rep
df_fake=pd.read_csv('Fake.csv')
df_fake['label']='Fake'
df_final=pd.concat([df_true,df_fake])
df_final=df_final.drop(['subject','date'], axis=1)
df_fake

In [ ]:
def findlabel(newtext):
    vec_newtest=tfidf_vectorizer.transform([newtext])
    y_pred1=pac.predict(vec_newtest)
    return y_pred1[0]

In [ ]:
findlabel((df_true['text'][0]))

In [ ]:
sum([1 if findlabel((df_true['text'][i]))=='Real' else 0 for i in range(len(df_true['text']))])/df_true['text'].size

In [ ]:
sum([1 if findlabel((df_fake['text'][i]))=='Fake' else 0 for i in range(len(df_fake['text']))])/df_fake['text'].size